In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
!pip install pyngrok scipy cloudinary

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-htp_8h4j/audiocraft_8d33e047873c4a9c9ec77d1cff06ea0e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-htp_8h4j/audiocraft_8d33e047873c4a9c9ec77d1cff06ea0e
  Resolved https://github.com/facebookresearch/audiocraft to commit 72cb16f9fb239e9cf03f7bd997198c7d7a67a01c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 102.8 MB/s eta 0:00:00
  Preparin

In [3]:
import cloudinary

cloudinary.config(
    cloud_name="depelknnb",
    api_key="325767256569419",
    api_secret="17mMfVn4Ju6K5_cx-rBvhbhsb20",
    secure=True
)

In [4]:
!ngrok authtoken '2gxa0prpElPzMMdSFhBRFBLdle6_7VEus1jY9kkDeagDKqdTa'


Authtoken saved to configuration file: /home/sagemaker-user/.config/ngrok/ngrok.yml                 


In [4]:
# !pip install torchaudio

In [6]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
app = Flask(__name__)

from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
# Initialize the model and set it to use CUDA if available
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = musicgen.MusicGen.get_pretrained('small', device=device)
model.set_generation_params(duration=8)  # Set duration to 8 seconds

False


/opt/conda/lib/python3.10/site-packages/audiocraft/models/musicgen.py:80: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
import numpy as np
import scipy.io.wavfile
import torch

def save_tensor_to_audio(audio_tensor, file_path, sample_rate=32000):
    # Convert tensor to numpy array
    audio_np = audio_tensor.cpu().numpy()
    print(audio_np)
    # Check if the data is floating point and scale to int16

    # Save the numpy array as a WAV file
    scipy.io.wavfile.write(file_path, rate=sample_rate, data=audio_np[0])
    print(f'Saved: {file_path}')

In [11]:
import cloudinary
import cloudinary.uploader

def generate_audio_from_text(text):
    print(f'text{text}')
    upload_results = []

    # Generate audio based on the input text
    results = model.generate(text, progress=False)
    # display_audio(results, 32000)
    for index, audio in enumerate(results):
        # Convert audio tensor to an audio file
        file_name = f"{text[index].replace(' ', '_')}_{index}.mp3"
        # Save the tensor as an audio file
        print(file_name)

        save_tensor_to_audio(audio, file_name, 32000)
        upload_result = cloudinary.uploader.upload(file_name, resource_type="video", public_id=file_name)
        upload_results.append(upload_result['secure_url'])

    return jsonify({'urls': upload_results})

classic_reggae_track_with_an_electronic_guitar_solo_0.mp3
[[-0.11885539 -0.12825061 -0.11344626 ...  0.02151875  0.02074005
   0.02510899]]
Saved: classic_reggae_track_with_an_electronic_guitar_solo_0.mp3
{'urls': ['https://res.cloudinary.com/depelknnb/video/upload/v1718332095/classic_reggae_track_with_an_electronic_guitar_solo_0.mp3.wav']}


In [14]:
# You can reload this cell to restart the server if you make changes

default_port = 6060

from werkzeug.serving import make_server
from flask import Flask
import threading

class ServerThread(threading.Thread):

    def __init__(self, app, port):
        threading.Thread.__init__(self)
        self.port = port
        self.srv = make_server('127.0.0.1', port, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print('starting server on port:',self.port)
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

def start_server(port=default_port):
    global server
    if 'server' in globals() and server:
      print('stopping server')
      stop_server()

    app = Flask('myapp')


    # you can add your own routes here as needed
    @app.route("/")
    def hello():
      # A wee bit o'html
      return '<h1 style="color:red;">Hello From Flask!</h1>'

    @app.route('/api/musics/generate/MUSIC_GEN', methods=['POST'])
    def generate_audio():
      # Retrieve text from the payload
      data = request.get_json()
      text_input = data['prompts']

      # Assume 'generate_audio_from_text' is a function you defined to interface with Audiocraft
      audio_url = generate_audio_from_text(text_input)

      return audio_url

    server = ServerThread(app,port)
    server.start()

def stop_server():
    global server
    if server:
      server.shutdown()
      server = None

# Start the server here
start_server()

ngrok_tunnel = ngrok.connect(default_port)
print('Public URL:', ngrok_tunnel.public_url)

stopping server
starting server on port: 6060
Public URL: https://b092-35-234-162-134.ngrok-free.app


In [12]:
# with app.app_context():
#     urls = generate_audio_from_text([
#         'classic reggae track with an electronic guitar solo',
#     ])
#     print(urls.json)

text['classic reggae track with an electronic guitar solo']
classic_reggae_track_with_an_electronic_guitar_solo_0.mp3
[[-0.07795804 -0.07934213 -0.06974236 ...  0.04259582  0.04089179
   0.04482312]]
Saved: classic_reggae_track_with_an_electronic_guitar_solo_0.mp3
{'urls': ['https://res.cloudinary.com/depelknnb/video/upload/v1718333113/classic_reggae_track_with_an_electronic_guitar_solo_0.mp3.wav']}


In [ ]:
# ngrok.disconnect(ngrok_tunnel.public_url)
# ngrok.kill()